# Load Python Modules

In [1]:
import warnings
warnings.simplefilter(action='ignore')

In [2]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import os
import numpy as np
import pandas as pd
import shutil
import deepchem as dc
import json

from rdkit import Chem
from sklearn.metrics import roc_auc_score
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm import tnrange, tqdm_notebook

/pylon5/ac5616p/yuke/anaconda3/envs/MolEnv2/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
os.chdir('/home/yuke/PythonProject/DrugEmbedding/')
from decode import *

In [4]:
"""
SIDER dataset loader.
"""
def load_sider(featurizer='ECFP', split='index', seed=0):
    #current_dir = os.path.dirname(os.path.realpath(__file__))

    # Load SIDER dataset
    print("About to load SIDER dataset.")
    dataset_file = os.path.join("./data/sider/deepchem/sider.csv.gz")
    dataset = dc.utils.save.load_from_disk(dataset_file)
    print("Columns of dataset: %s" % str(dataset.columns.values))
    print("Number of examples in dataset: %s" % str(dataset.shape[0]))

    # Featurize SIDER dataset
    print("About to featurize SIDER dataset.")
    if featurizer == 'ECFP':
        featurizer = dc.feat.CircularFingerprint(size=1024)
    elif featurizer == 'GraphConv':
        featurizer = dc.feat.ConvMolFeaturizer()

    SIDER_tasks = dataset.columns.values[1:].tolist()
    print("SIDER tasks: %s" % str(SIDER_tasks))
    print("%d tasks in total" % len(SIDER_tasks))

    loader = dc.data.CSVLoader(
      tasks=SIDER_tasks, smiles_field="smiles", featurizer=featurizer)
    dataset = loader.featurize(dataset_file)
    print("%d datapoints in SIDER dataset" % len(dataset))

    # Initialize transformers
    transformers = [
      dc.trans.BalancingTransformer(transform_w=True, dataset=dataset)]
    print("About to transform data")
    for transformer in transformers:
        dataset = transformer.transform(dataset)

    splitters = {'index': dc.splits.IndexSplitter(),
               'random': dc.splits.RandomSplitter(),
               'scaffold': dc.splits.ScaffoldSplitter()}
    splitter = splitters[split]
    train, valid, test = splitter.train_valid_test_split(dataset, seed=seed)

    return SIDER_tasks, (train, valid, test), transformers

# Load Model

In [435]:
#seed = 100, 99, 98, 97, 95
seed = 101
np.random.seed(seed)

In [436]:
exp_dir = './experiments/KDD/kdd_010'
checkpoint = 'checkpoint_epoch110.model'
#exp_dir = './experiments/EXP_TASK/exp_task_010'
#checkpoint = 'checkpoint_epoch100.model'
config_path = os.path.join(exp_dir, 'configs.json')
checkpoint_path = os.path.join(exp_dir, checkpoint)

with open(config_path, 'r') as fp:
    configs = json.load(fp)
fp.close()

configs['checkpoint'] = checkpoint
configs

{'data_dir': './data/fda_drugs',
 'data_file': 'smiles_set_clean.smi',
 'fda_file': 'all_drugs.smi',
 'vocab_file': 'char_set_clean.pkl',
 'atc_sim_file': 'drugs_sp_all.csv',
 'checkpoint_dir': './experiments/KDD',
 'experiment_name': 'kdd_010',
 'task': 'vae + atc',
 'limit': 0,
 'batch_size': 128,
 'epochs': 100,
 'max_sequence_length': 120,
 'learning_rate': 0.0003,
 'max_norm': 1000000000000.0,
 'wd': 0.0,
 'manifold_type': 'Lorentz',
 'prior_type': 'Standard',
 'num_centroids': 0,
 'bidirectional': False,
 'num_layers': 1,
 'hidden_size': 512,
 'latent_size': 64,
 'word_dropout_rate': 0.2,
 'anneal_function': 'logistic',
 'k': 0.51,
 'x0': 29.0,
 'C': 1.0,
 'num_workers': 4,
 'logging_steps': 1,
 'save_per_epochs': 10,
 'new_training': False,
 'new_annealing': False,
 'checkpoint': 'checkpoint_epoch110.model',
 'trained_epochs': 110,
 'alpha': 0.0,
 'beta': 0.015625,
 'gamma': 0.0,
 'delta': 11.0,
 'nneg': 11,
 'fda_prop': 0.2}

In [437]:
model = load_model(configs)
print(model)

HVAE(
  (encoder_rnn): GRU(49, 512, batch_first=True)
  (decoder_rnn): GRU(49, 512, batch_first=True)
  (hidden2mean): Linear(in_features=512, out_features=64, bias=True)
  (hidden2logv): Linear(in_features=512, out_features=64, bias=True)
  (latent2hidden): Linear(in_features=65, out_features=512, bias=True)
  (outputs2vocab): Linear(in_features=512, out_features=49, bias=True)
  (RECON): NLLLoss()
)


In [438]:
exp_dir = './experiments/KDD/kdd_009'
checkpoint = 'checkpoint_epoch110.model'
#exp_dir = './experiments/EXP_TASK/exp_task_009'
#checkpoint = 'checkpoint_epoch100.model'
config_path = os.path.join(exp_dir, 'configs.json')
checkpoint_path = os.path.join(exp_dir, checkpoint)

with open(config_path, 'r') as fp:
    configs_e = json.load(fp)
fp.close()

configs_e['checkpoint'] = checkpoint
configs_e

{'data_dir': './data/fda_drugs',
 'data_file': 'smiles_set_clean.smi',
 'fda_file': 'all_drugs.smi',
 'vocab_file': 'char_set_clean.pkl',
 'atc_sim_file': 'drugs_sp_all.csv',
 'checkpoint_dir': './experiments/KDD',
 'experiment_name': 'kdd_009',
 'task': 'vae + atc',
 'limit': 0,
 'batch_size': 128,
 'epochs': 100,
 'max_sequence_length': 120,
 'learning_rate': 0.0003,
 'max_norm': 1000000000000.0,
 'wd': 0.0,
 'manifold_type': 'Euclidean',
 'prior_type': 'Standard',
 'num_centroids': 0,
 'bidirectional': False,
 'num_layers': 1,
 'hidden_size': 512,
 'latent_size': 64,
 'word_dropout_rate': 0.2,
 'anneal_function': 'logistic',
 'k': 0.51,
 'x0': 29.0,
 'C': 1.0,
 'num_workers': 4,
 'logging_steps': 1,
 'save_per_epochs': 10,
 'new_training': False,
 'new_annealing': False,
 'checkpoint': 'checkpoint_epoch110.model',
 'trained_epochs': 110,
 'alpha': 0.0,
 'beta': 0.015625,
 'gamma': 0.0,
 'delta': 11.0,
 'nneg': 11,
 'fda_prop': 0.2}

In [439]:
model_e = load_model(configs_e)
print(model_e)

EVAE(
  (encoder_rnn): GRU(49, 512, batch_first=True)
  (decoder_rnn): GRU(49, 512, batch_first=True)
  (hidden2mean): Linear(in_features=512, out_features=64, bias=True)
  (hidden2logv): Linear(in_features=512, out_features=64, bias=True)
  (latent2hidden): Linear(in_features=64, out_features=512, bias=True)
  (outputs2vocab): Linear(in_features=512, out_features=49, bias=True)
  (RECON): NLLLoss()
)


# Process Data

In [440]:
sider_tasks, datasets, transformers = load_sider(split='random', seed=seed)
train_dataset, valid_dataset, test_dataset = datasets

About to load SIDER dataset.
Columns of dataset: ['smiles' 'Hepatobiliary disorders' 'Metabolism and nutrition disorders'
 'Product issues' 'Eye disorders' 'Investigations'
 'Musculoskeletal and connective tissue disorders'
 'Gastrointestinal disorders' 'Social circumstances'
 'Immune system disorders' 'Reproductive system and breast disorders'
 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)'
 'General disorders and administration site conditions'
 'Endocrine disorders' 'Surgical and medical procedures'
 'Vascular disorders' 'Blood and lymphatic system disorders'
 'Skin and subcutaneous tissue disorders'
 'Congenital, familial and genetic disorders'
 'Infections and infestations'
 'Respiratory, thoracic and mediastinal disorders' 'Psychiatric disorders'
 'Renal and urinary disorders'
 'Pregnancy, puerperium and perinatal conditions'
 'Ear and labyrinth disorders' 'Cardiac disorders'
 'Nervous system disorders'
 'Injury, poisoning and procedural complications']
Num

In [441]:
# side effects labels
train_y = train_dataset.y
valid_y = valid_dataset.y
test_y = test_dataset.y

In [442]:
# Finger Print
train_fp = train_dataset.X
valid_fp = valid_dataset.X
test_fp = test_dataset.X

In [443]:
train_smi = train_dataset.ids
valid_smi = valid_dataset.ids
test_smi = test_dataset.ids

In [444]:
def smi2mean(configs, model, smi_lst):
    mean_lst = []
    for i in tnrange(len(smi_lst)):
        # convert to canonical form
        smi = smi_lst[i]
        try:
            smi_can = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
        except:
            smi_can = smi
        mean, _ = smiles2mean(configs, smi_can, model)
        mean_lst.append(mean.squeeze().cpu().detach().numpy())
    return np.array(mean_lst)

In [445]:
train_mu = smi2mean(configs, model, train_smi)

In [446]:
valid_mu = smi2mean(configs, model, valid_smi)

In [447]:
test_mu = smi2mean(configs, model, test_smi)

In [448]:
train_mu_e = smi2mean(configs_e, model_e, train_smi)

In [449]:
valid_mu_e = smi2mean(configs_e, model_e, valid_smi)

In [450]:
test_mu_e = smi2mean(configs_e, model_e, test_smi)

# Custom Distance Functions

In [451]:
def lor_dist(z1, z2):
    m = z1*z2
    lor_prod = m[1:].sum() - m[0]
    x = - lor_prod
    x = np.where(x<1.0, 1.0+1e-6, x)
    return np.log(x + np.sqrt(x**2 - 1))

# KNN Classifier

In [452]:
def get_scores(y_true, y_pred):
    roc_aucs = []
    n_data, n_dim = y_true.shape
    for j in range(n_dim):
        true_label = y_true[:, j]
        pred_label = y_pred[:, j]
        roc_aucs.append(roc_auc_score(true_label, pred_label))
    return np.array(roc_aucs)

In [456]:
N_val = 11
W_val = 'distance'

## Fit and Predict

### Finger Prints

In [457]:
# Finger Print Representations
neigh_fp = KNeighborsClassifier(n_neighbors=N_val, algorithm='brute', metric='rogerstanimoto', weights=W_val)
neigh_fp.fit(train_fp, train_y)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='rogerstanimoto',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='distance')

In [458]:
pred_fp = neigh_fp.predict_proba(test_fp) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_pred_fp = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_pred_fp[:, j] = pred_fp[j][:, 1]
    
#test_y_pred_fp = neigh_fp.predict(test_fp)

### Lorentz Embeddings

In [459]:
neigh_lor = KNeighborsClassifier(n_neighbors=N_val, algorithm='brute', metric=lor_dist, weights=W_val)
neigh_lor.fit(train_mu, train_y)

KNeighborsClassifier(algorithm='brute', leaf_size=30,
                     metric=<function lor_dist at 0x7ef0981f8ae8>,
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='distance')

In [460]:
pred_lor = neigh_lor.predict_proba(test_mu) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_pred_lor = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_pred_lor[:, j] = pred_lor[j][:, 1]    
#test_y_pred_lor = neigh_lor.predict(test_mu)

### Euclidean Embeddings

In [461]:
neigh_euc = KNeighborsClassifier(n_neighbors=N_val, algorithm='brute', weights=W_val)
neigh_euc.fit(train_mu_e, train_y)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='distance')

In [462]:
pred_euc = neigh_euc.predict_proba(test_mu_e) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_pred_euc = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_pred_euc[:, j] = pred_euc[j][:, 1]    
#test_y_pred_lor = neigh_lor.predict(test_mu)

## Evaluation

In [463]:
get_scores(test_y, test_y_pred_fp).mean()

0.6207228576607152

In [464]:
get_scores(test_y, test_y_pred_lor).mean()

0.6339681634232448

In [465]:
get_scores(test_y, test_y_pred_euc).mean()

0.7001248119569888

# RF

## Finger Print

In [466]:
rf_fp = RandomForestClassifier(class_weight="balanced", n_estimators=500)
rf_fp.fit(train_fp, train_y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [467]:
rf_pred_fp = rf_fp.predict_proba(test_fp) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_rf_pred_fp = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_rf_pred_fp[:, j] = rf_pred_fp[j][:, 1]    
#test_y_pred_lor = neigh_lor.predict(test_mu)

In [468]:
get_scores(test_y, test_y_rf_pred_fp).mean()

0.6646586257362254

## Lorentz Embeddings

In [469]:
rf_lor = RandomForestClassifier(class_weight="balanced", n_estimators=500)
rf_lor.fit(train_mu, train_y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [470]:
rf_pred_lor = rf_lor.predict_proba(test_mu) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_rf_pred_lor = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_rf_pred_lor[:, j] = rf_pred_lor[j][:, 1]    
#test_y_pred_lor = neigh_lor.predict(test_mu)

In [471]:
get_scores(test_y, test_y_rf_pred_lor).mean()

0.6571116170898003

## Euclidean Embeddings

In [472]:
rf_euc = RandomForestClassifier(class_weight="balanced", n_estimators=500)
rf_euc.fit(train_mu_e, train_y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [473]:
rf_pred_euc = rf_euc.predict_proba(test_mu_e) # 27 * size(x) * 2

# covert predicted probability to an array
n_data, n_dim = test_y.shape
test_y_rf_pred_euc = np.zeros((n_data, n_dim))
for j in range(n_dim):
    test_y_rf_pred_euc[:, j] = rf_pred_euc[j][:, 1]    
#test_y_pred_lor = neigh_lor.predict(test_mu)

In [474]:
get_scores(test_y, test_y_rf_pred_euc).mean()

0.6813400280403821

# Deepchem Models

In [475]:
"""
Script that trains Sklearn multitask models on the sider dataset
@Author Bharath Ramsundar, Aneesh Pappu
"""
from sklearn.ensemble import RandomForestClassifier

metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean,
                           mode="classification")

def model_builder(model_dir):
    sklearn_model = RandomForestClassifier(class_weight="balanced", n_estimators=100)
    return dc.models.SklearnModel(sklearn_model, model_dir)

model = dc.models.SingletaskToMultitask(sider_tasks, model_builder)

# Fit trained model
model.fit(train_dataset)
model.save()

print("About to evaluate model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)

print("Test scores")
print(test_scores)

About to initialize singletask to multitask model
Initializing directory for task Hepatobiliary disorders
Initializing directory for task Metabolism and nutrition disorders
Initializing directory for task Product issues
Initializing directory for task Eye disorders
Initializing directory for task Investigations
Initializing directory for task Musculoskeletal and connective tissue disorders
Initializing directory for task Gastrointestinal disorders
Initializing directory for task Social circumstances
Initializing directory for task Immune system disorders
Initializing directory for task Reproductive system and breast disorders
Initializing directory for task Neoplasms benign, malignant and unspecified (incl cysts and polyps)
Initializing directory for task General disorders and administration site conditions
Initializing directory for task Endocrine disorders
Initializing directory for task Surgical and medical procedures
Initializing directory for task Vascular disorders
Initializing d

Fitting model for task General disorders and administration site conditions
Fitting model for task Endocrine disorders
Fitting model for task Surgical and medical procedures
Fitting model for task Vascular disorders
Fitting model for task Blood and lymphatic system disorders
Fitting model for task Skin and subcutaneous tissue disorders
Fitting model for task Congenital, familial and genetic disorders
Fitting model for task Infections and infestations
Fitting model for task Respiratory, thoracic and mediastinal disorders
Fitting model for task Psychiatric disorders
Fitting model for task Renal and urinary disorders
Fitting model for task Pregnancy, puerperium and perinatal conditions
Fitting model for task Ear and labyrinth disorders
Fitting model for task Cardiac disorders
Fitting model for task Nervous system disorders
Fitting model for task Injury, poisoning and procedural complications
About to evaluate model
computed_metrics: [1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 

# Results

In [11]:
#seed = 100, 99, 98, 97, 95

knn_fp_auc = [ 0.6489, 0.6077, 0.6403, 0.5954, 0.6458]
knn_lor_auc = [0.6752, 0.6724, 0.6710, 0.6691, 0.6919]
knn_euc_auc = [0.6815, 0.6887, 0.6857, 0.6804, 0.7213]
rf_fp_auc = [0.6826, 0.6684, 0.6665, 0.6783, 0.6497]
rf_lor_auc = [0.7123, 0.7227, 0.6968, 0.6928, 0.6983]
rf_euc_auc = [0.7099, 0.7018, 0.6825, 0.6906, 0.6905]

# no ATC information, only chemical structures
rf_lor_chem_auc = [0.6431, 0.6387, 0.6093, 0.5854, 0.6375]
rf_euc_chem_auc = [0.6359, 0.6140, 0.6292, 0.5955, 0.6215]

In [12]:
rf_fp_arr = np.array(rf_fp_auc)
rf_lor_arr = np.array(rf_lor_auc)
rf_euc_arr = np.array(rf_euc_auc)

In [13]:
print('FP mean:' + str(rf_fp_arr.mean()) + ', std:' + str(rf_fp_arr.std()))

FP mean:0.6691, std:0.011406138698087077


In [14]:
print('Lorentz mean:' + str(rf_lor_arr.mean()) + ', std:' + str(rf_lor_arr.std()))

Lorentz mean:0.7045800000000001, std:0.01119524899231814


In [15]:
print('Euclidean mean:' + str(rf_euc_arr.mean()) + ', std:' + str(rf_euc_arr.std()))

Euclidean mean:0.69506, std:0.009634230638717335


# Stats

In [16]:
from scipy import stats

## Paired T-test between RF+FP and RF+LDE

In [17]:
stats.ttest_rel(rf_fp_arr,rf_lor_arr)

Ttest_relResult(statistic=-4.951889056380475, pvalue=0.0077507605194669415)

## Paired T-test between RF+LDE and RF+EUC

In [18]:
stats.ttest_rel(rf_euc_arr,rf_lor_arr)

Ttest_relResult(statistic=-2.6424674113529942, pvalue=0.05743200947957062)